#### *Run this cell to resolve import issues*

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

#### Imports and variables

In [8]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from log_parser import parse_keyboard_log
from numpy_ringbuffer import RingBuffer

# Size of sliding window for each key 
# (mean and stddev of the last {sample_size} key taps)
sample_size = 20

# Amount of data to record before overwriting old data
data_window_size = 100

# Read logfile paths from .routing
log_paths = None

#### Data persistent functions

In [9]:
# Save data to file in profiles directory
def save_tap_data(profile, data):
    with open(f"profiles/{profile}.txt", 'w', encoding='utf-8') as f:
        keys = list(data.keys())      # List keys that have data associated with them
        for key in keys:              # For each key
            values = list(data[key])  # List data values
            f.write(str(key) + ',' + str(data[key].shape[0]) + ',') # Write key and #values
            f.write(','.join(format(x, "0.4f") for x in values))    # Write values
            f.write('\n')                                           # Formatting


# Load data from saved profile and return a dict
def load_tap_data (profile):
    duration_data = {}  # Dict to store data

    with open(f"profiles/{profile}.txt", 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()   # Read lines without '\n's
        for line in lines:
            data = line.split(',') # Separate values by commas

            # Create key and initialize ring buffer
            key = data[0]          # First value is the key
            duration_data[key] = RingBuffer(
                capacity = data_window_size * 2,
                dtype = float )
            for value in data[2:]:  # Append following values to the ring buffer
                duration_data[key].append(value)
    
    return duration_data


### Gather key tap duration data from log files
Records all quick presses ("taps") of keys, calculate the mean and stddev of the last
{sample_size} taps, and store the data in a larger sliding window of size {data_window_size}.

*Currently the last 20 taps are analyzed and the last 100 are used to train the user's profile.*

---

In [10]:
# Read logfile paths from .routing
with open("../.routing", 'r', encoding='utf-8') as f:
    log_paths = f.read().splitlines()   # Read lines without '\n's

#if log_paths == None: handle error or something

for log_path in log_paths: # For each log file

    if "key.log" not in log_path: continue      # Make sure it's a keylog
    profile = ''.join(log_path.split('/')[2:4]) # Concatenate profile+character dir names
    kb = parse_keyboard_log("../" + log_path)   # Convert keylog to DataFrame

    durations_dict = {} # Map each key to a ring buffer of size {sample_size}
    presstimes     = {} # Record the 'last pressed' time of each key
    duration_data  = {} # Records the mean/stddev of each key sample window
    
    for _, row in kb.iterrows():    # Analyze each piece of data one at a time
        key = str(row.key).replace('\'', '') # Get rid of ' characters

        if key not in durations_dict.keys():   # If key not added to dict yet
            durations_dict[key] = RingBuffer(capacity=sample_size, dtype=float) # Add it
        
        if row.action == 'pressed':     # If key was pressed
            presstimes[key] = row.time  # Record time of press
        
        # If key was released and it has been pressed before
        elif row.action == 'released' and key in presstimes.keys():
            duration = row.time - presstimes[key] # Calculate press duration

            if duration < 0.4: # Only append key taps (<0.4s)
                durations_dict[key].append(duration) # Append tap duration to ring buffer

                durations_np = np.array(durations_dict[key]) # Convert to numpy array
                if durations_np.size > 3:   # If more than 3 samples in the ring buffer
                    
                    if key not in duration_data.keys():   # If key not added to data dict yet
                        duration_data[key] = RingBuffer(  # Add it
                            capacity = data_window_size * 2,
                            dtype = float )

                    # Interleave data (mean0,std0,mean1,std1,mean2,std2,...)
                    duration_data[key].append(durations_np.mean())
                    duration_data[key].append(durations_np.std()) 

    # Write data to file
    save_tap_data(profile, duration_data)

